# Packages

In [1]:
using Oscar
import LinearAlgebra as Lin
import HomotopyContinuation as HC
import Combinatorics as CB

# Rank Condition Check

Given $N\in\mathbb{Q}^{s\times r}$ and $B\in\mathbb{Z}^{n\times r}$, following function computes the rank of $N{\rm diag}(Gp)B^{\top}$ for a random $p\in\mathbb{Z}^{r-s}$. If the rank is $s$, then we may conclude that will be generically of expected dimension $n-s$ hence non-empty.  

In [2]:
function compute_rank(N::QQMatrix, B::ZZMatrix)
    s=size(N)[1];
    r=size(N)[2];
    n=size(B)[1];
    p=rand(-100:100,r-s)
    G=nullspace(N)[2]
    D=diagonal_matrix(G*p)
    return rank(N*D*transpose(B))
end

compute_rank (generic function with 1 method)

In [3]:
N=matrix(QQ,[1 1 1 0 0 0 0 0 0; 0 0 0 1 1 1 0 0 0; 0 0 0 0 0 0 1 1 1])
B=matrix(ZZ,[1 0 0 1 1 0 1 2 0; 0 1 0 0 1 0 0 1 0; 0 1 0 0 1 0 0 1 0])

[1   0   0   1   1   0   1   2   0]
[0   1   0   0   1   0   0   1   0]
[0   1   0   0   1   0   0   1   0]

The following function takes the matrices $N\in\mathbb{Q}^{s\times r}$ and $B\in\mathbb{Z}^{n\times r}$ as input and returns the boolean  "true" if all the $s\times s$ minors of $N{\rm diag}(Gu)B^{\top}\in\mathbb{Q}[u]^{r-s}$ vanish. It returns the boolean "false" otherwise.

In [3]:
function minors_symbolic_matrix(N::QQMatrix, B::ZZMatrix)
    s=size(N)[1];
    r=size(N)[2];
    n=size(B)[1];
    T,u=polynomial_ring(QQ,"u"=>1:r-s)
    G=nullspace(N)[2]
    D=diagonal_matrix(G*u)
    A=N*D*transpose(B)
    return all(m->iszero(m),minors(A,s))   
end 

minors_symbolic_matrix (generic function with 1 method)

In [4]:
function symbolic_matrix(N::QQMatrix, B::ZZMatrix)
    s=size(N)[1];
    r=size(N)[2];
    n=size(B)[1];
    T,u=polynomial_ring(QQ,"u"=>1:r-s)
    G=nullspace(N)[2]
    D=diagonal_matrix(G*u)
    A=N*D*transpose(B)
    return A 
end 

symbolic_matrix (generic function with 1 method)

# Computing Generic Root Count of Square Subsystems

We create a function that takes a QQfield-vector and turns it into a Rational{Int64}-vector. We will need this to define systems and solve them using HomotopyContinuation, as QQ-field elements may not be directly multiplied by HomotopyContinuation expressions.

In [5]:
function toRatInt64(v::QQMatrix)
    res=Rational{Int64}[]
    for i in 1:length(v)
        res=push!(res,Int64(numerator(v[i]))//Int64(denominator(v[i])))
    end
    return res
end 

toRatInt64 (generic function with 1 method)

We create a function that, given $s$ and $k$ with $k\leq s$, returns a list of lists containing all subsets of $[s]$ with $k$ elements.

In [6]:
function subs(s::Int64,k::Int64)
    set=1:s
    return collect(CB.combinations(set,k))
end

subs (generic function with 1 method)

The following function receives $N\in \mathbb{Q}^{s\times r}$, $B\in\mathbb{Z}^{n\times r}$ and $S\subseteq [s]$ as input, and returns a three-entry vector. The first entry is a submatrix of $B$ containing the columns (that is, a collection of exponent vectors) that correspond to the monomials appearing in the subsystem of 
$$f=\begin{pmatrix} f_1 \\f_2\\ \vdots \\ f_s    \end{pmatrix}=N(x^B)\in\mathbb{Q}[x^{\pm}]^s $$
given by the polynomials indexed by elements in $S$. The second entry is a list consisting of the indeces that these selected columns have in matrix $B$. The third entry is $S$.

In [7]:
function select_columns(N::QQMatrix,B::ZZMatrix,S::Vector{Int64})
    lista=[]
    k=length(S)
    r=size(N)[2]
    n=size(B)[1]
       N_sub=N[S,collect(1:r)]
       for i in 1:k
            for l in 1:r
                if N_sub[i,l]!=0
                    lista=push!(lista,l)
                end
            end
        end 
    list_no_rep=collect(Set(lista))
    list_no_rep=[list_no_rep[i] for i in 1:length(list_no_rep)]
    return [B[collect(1:n),list_no_rep],list_no_rep,S]  
end 

select_columns (generic function with 1 method)

The following function collects $k\times k$ square subsystems: it receives $N\in \mathbb{Q}^{s\times r}$, $B\in\mathbb{Z}^{n\times r}$ and $k\leq n$ and returns a vector of 3-entry vectors consisting of the same data as the output of select_columns(). Each 3-entry vector on this list encodes an induced $k\times k$ square subsystem of $f$

In [8]:
function induced_square_subsystems(N::QQMatrix,B::ZZMatrix,k::Int64)
    s=size(N)[1]
    res=[]
        for S in subs(s,k)
            B_sub,L,S=select_columns(N,B,S)
            if rank(B_sub)==k
              res=push!(res,[B_sub,L,S])
            end 
        end      
        return res
end 

induced_square_subsystems (generic function with 1 method)

The following function takes $N\in \mathbb{Q}^{s\times r}$, $B\in\mathbb{Z}^{n\times r}$ as input and computes all the induced square subsystems $(f_j)_{j\in S}$ of $f$ using induced_square_subsystems():


 For each $S\subseteq [s]$ with $|S|=k$, it makes the appropriate transformation to turn them into actual square subsystems by means of the Smith Normal Form and  returns a vector of two-entry vectors. Each one of these two-entry vectors consists of the following: 
 
 -The first entry is a submatrix of $N$ comprised of the rows indexed by $S$, and the columns corresponding to the monomials involved in $(f_j)_{j\in S}$, say they are indexed by $L\subseteq [r]$. 
 
 The transformation $\tilde{B}\coloneqq TB$ of $B$ satisfies that the last $n-k$ entries of the columns of $\tilde{B}$ indexed by $L$ are all zero. 
 
 -The second entry is the submatrix of $\tilde{B}$ given by the first $k$ rows and the columns indexed by $L$.

In [9]:
function square_subsystems(N::QQMatrix,B::ZZMatrix)
    res=[]
    s=size(N)[1]
    for k in 1:s
        L=induced_square_subsystems(N,B,k)
        for m in 1:length(L)
             N_sub=N[L[m][3], L[m][2]]
             P,T,D = snf_with_transform(L[m][1])
             B_transf=Int64.(Matrix(T*B))
             B_transf_sub=B_transf[collect(1:k), L[m][2]]
             res=push!(res,[N_sub,B_transf_sub])
        end
    end 
    
    return res
end 

square_subsystems (generic function with 1 method)

In [10]:
function positiveExp(Q)
    for i in 1:size(Q)[1]
        if any(x -> x < 0, Q[i,:])
            m=minimum(Q[i,:])
            Q[i,:].=Q[i,:].+(-m)   
        end
    end
    return Q
end 



positiveExp (generic function with 1 method)

Let $P\in \mathbb{Q}^{k\times r}$ and $Q\in\mathbb{Z}^{k\times r}$. The following function computes the generic root count of the parametric polynomial system
$$ P(\kappa\circ x^Q)\in\mathbb{C}[\kappa_1,\dots,\kappa_r,x_1^{\pm},\dots,x_k^{\pm}]^k $$
by solving $$ P(a\circ x^Q)\in\mathbb{Q}[x_1^{\pm},\dots,x_k^{\pm}]^k $$
for a random $a\in\mathbb{Q}^r$. 

In [11]:
function grc(P::QQMatrix,Q::Matrix{Int64})
    if (size(P)[1]==size(Q)[1])
        Q=positiveExp(Q)
        k=size(Q)[1]
        r=size(Q)[2]
        a = HC.rand(-100:100,r)
        P=P*diagonal_matrix(a)
        HC.@var y[1:k]
        f=[Lin.dot(toRatInt64(P[i,:]),[prod([y[j]^(Q[j,l]) for j in 1:k]) for l in 1:r]) for i in 1:k]
        G=HC.System(f)
        return length(HC.solutions(HC.solve(G,only_non_zero=true)))
    else
        return error("Error: not a square system")
    end 
    
end 

grc (generic function with 1 method)

The following function receives matrices $N\in \mathbb{Q}^{s\times r}$, $B\in\mathbb{Z}^{n\times r}$ and return a list containing the generic root count of each of the square subsystems of  
$$ N(\kappa\circ x^B)\in\mathbb{Q}[\kappa,x^{\pm}]^s. $$

In [12]:
function list_of_grc(N::QQMatrix,B::ZZMatrix)
    res=[]
    L=square_subsystems(N,B)
    for i in 1:length(L)
        res=push!(res,grc(L[i][1],L[i][2]))
    end 
    return res
end     

list_of_grc (generic function with 1 method)

# Irreducibility Check 

The following functions takes $N\in\mathbb{Q}^{s\times r}$ and $B\in\mathbb{Z}^{n\times r}$ as input and returns the polynomial system:
$$ N (a\circ x^{B})\in\mathbb{Q}[x_1^{\pm},\dots,x_n^{\pm}]^s, $$
for some random $a\in\mathbb{Q}^r$.

In [13]:
function system_OS(N::QQMatrix,B::ZZMatrix)
    n=size(B)[1]
    s=size(N)[1]
    r=size(B)[2]
    a=rand(-100:100,r)
    N_1=Matrix(N)*Lin.diagm(a)
    local S,x=polynomial_ring(QQ,"x"=>1:n)
    f=[dot(N_1[i,collect(1:r)],([prod([x[j]^Int64(numerator(B[j,l])) for j in 1:n]) for l in 1:r])) for i in 1:s]
    return f
end 

system_OS (generic function with 1 method)

In [14]:
function system_OS_ones(N::QQMatrix,B::ZZMatrix)
    n=size(B)[1]
    s=size(N)[1]
    r=size(B)[2]
    N_1=Matrix(N)
    local S,x=polynomial_ring(QQ,"x"=>1:n)
    f=[dot(N_1[i,collect(1:r)],([prod([x[j]^Int64(numerator(B[j,l])) for j in 1:n]) for l in 1:r])) for i in 1:s]
    return f
end 

system_OS_ones (generic function with 1 method)

In [15]:
function system_HC(N::QQMatrix,B::ZZMatrix)
    n=size(B)[1]
    r=size(B)[2]
    s=size(N)[1]
    k=rand(-100:100,r)
    N_1=Matrix(N)*Lin.diagm(k)
    N_1=matrix(QQ,N_1)
    HC.@var y[1:n]
    f=[Lin.dot(toRatInt64(N_1[i,:]),[prod([y[j]^Int64(numerator(B[j,l])) for j in 1:n]) for l in 1:r]) for i in 1:s]
    return HC.System(f)
end 


system_HC (generic function with 1 method)


The following function takes $N\in\mathbb{Q}^{s\times r}$ and $B\in\mathbb{Z}^{n\times r}$ as input and returns the boolean true if the ideal in $\mathbb{C}[x^{\pm}] $ which generated by the system

$$ N (\kappa\circ x^{B})\in\mathbb{C}[x_1^{\pm},\dots,x_n^{\pm}]^s $$
 is prime for generic $\kappa\in\mathbb{C}^r$ and the boolean false otherwise.

Only to be tested for proper ideals. The function absolute_primary_decomposition runs into problems when dealing with the unit ideal.

In [16]:
function irred_check(N::QQMatrix,B::ZZMatrix)
    f=system_OS(N,B)
    R=parent(f[1])
    x=gens(R)
    # I=saturation(ideal(R,f), ideal(R,prod([x[i] for i in 1:length(x)])))
    I=ideal(R,f)
    L=absolute_primary_decomposition(I)
    if length(L)>1
        return false
    else 
        if L[1][4]!=1
            return false

        end
    end
    return true
end 

irred_check (generic function with 1 method)

In [17]:
function irred_check_numerical(N::QQMatrix,B::ZZMatrix)
    @assert size(N,1)<size(B,1) 
    G=system_HC(N,B)
    nid_result = HC.nid(G, show_progress=false)
    if HC.n_components(nid_result) == 0
        error("Found no irreducible components")
    elseif HC.n_components(nid_result) == 1
        return true
    else
        return false
    end
end 


irred_check_numerical (generic function with 1 method)

# Main Function

Given matrices $N\in\mathbb{Q}^{s\times r}$ and $B\in\mathbb{Z}^{n\times r}$ we consider the parametric polynomial system

$$ f=N (\kappa\circ x^{B})\in\mathbb{C}[\kappa_1,\dots,\kappa_r,x_1^{\pm},\dots,x_n^{\pm}]^s. $$

We make a function that returns a message depending on whether:


-the ideal $(f_{\kappa})\subseteq\mathbb{C}[x^{\pm}]$ is the unit ideal for generic $\kappa\in\mathbb{C}^r$. If it is, it return the message "The variety is generically empty";

-the ideal $(f_{\kappa})\subseteq\mathbb{C}[x^{\pm}]$ is prime for generic $\kappa\in\mathbb{C}^r$;

-the systems has square subsystems with generic root count greater than 1.

In [18]:
function main(N::QQMatrix,B::ZZMatrix)
    if compute_rank(N,B)<size(N)[1]
        if minors_symbolic_matrix(N,B)==true
            return  "The variety is generically empty."
        end 
    else
        list_of_generic_root_counts=list_of_grc(N,B)
        is_generically_irreducible=irred_check_numerical(N,B)
        all_gcr_are_one=all(list_of_generic_root_counts.==1)

        if (is_generically_irreducible && all_gcr_are_one)
            return "Supportive evidence that the conjecture is true: all square subsystems have generic root count 1 and the variety is generically irreducible"
        elseif (!(is_generically_irreducible) && !(all_gcr_are_one))
            return "Supportive evidence that the conjecture is true: there is a square subsystem with generic root count greater than 1 and the variety is generically reducible"
        elseif ((!(is_generically_irreducible) && all_gcr_are_one))
            return "Candidate for counterexample" 
        elseif (is_generically_irreducible && !(all_gcr_are_one))
            return "Discrepancy due to the non-deterministic nature of the method"
        end 
    end  
end


main (generic function with 1 method)

# Example 1


$$ \begin{cases}
     \kappa_1x_1+\kappa_2x_2 x_3+\kappa_3\\
     \kappa_4x_1+\kappa_5 x_1x_2x_3+\kappa_6\\
     \kappa_7 x_1+ \kappa_8 x_1^2 x_2x_3+\kappa_9\\ 
 \end{cases} $$

 We know that the variety is generically empty. We check it computationally.

In [19]:
N_1=matrix(QQ,[1 1 1 0 0 0 0 0 0; 0 0 0 1 1 1 0 0 0; 0 0 0 0 0 0 1 1 1])

B_1=matrix(ZZ,[1 0 0 1 1 0 1 2 0; 0 1 0 0 1 0 0 1 0; 0 1 0 0 1 0 0 1 0])


[1   0   0   1   1   0   1   2   0]
[0   1   0   0   1   0   0   1   0]
[0   1   0   0   1   0   0   1   0]

In [20]:
main(N_1,B_1)

"The variety is generically empty."

# Example 2



$$\begin{cases}
     \kappa_1x_1+\kappa_2x_2x_3 x_4+\kappa_3\\
     \kappa_4x_1+\kappa_5 x_1x_2x_3x_4+\kappa_6\\
     \kappa_7 x_3+\kappa_8 x_4^2+\kappa_9\\ 
     \end{cases} $$

We know it has only one induced square subsystem given by the first and second polynomials. And this square subsystem has generic root count 2. Since the system is freely parametrized we know that the variety will be generically reducible.

In [21]:
N_2=matrix(QQ,[1 1 1 0 0 0 0 0 0; 0 0 0 1 1 1 0 0 0;0 0 0 0 0 0 1 1 1])

B_2=matrix(ZZ,[1 0 0 1 1 0 0 0 0; 0 1 0 0 1 0 0 0 0; 0 1 0 0 1 0 1 0 0; 0 1 0 0 1 0 0 2 0])


[1   0   0   1   1   0   0   0   0]
[0   1   0   0   1   0   0   0   0]
[0   1   0   0   1   0   1   0   0]
[0   1   0   0   1   0   0   2   0]

In [22]:
main(N_2,B_2)

"Supportive evidence that the conjecture is true: there is a square subsystem with generic root count greater than 1 and the variety is generically reducible"

# Example 3



$$\begin{cases}
     \kappa_1x_1+\kappa_2x_2x_3 x_4+\kappa_3\\
     \kappa_4x_1+\kappa_5 x_2x_3x_4+\kappa_6\\
     \kappa_7 x_3+\kappa_8 x_4^2+\kappa_9\\ 
     \end{cases} $$

We know it has only one induced square subsystem given by the first and second polynomials. And this square subsystem has generic root count 1. Since the system is freely parametrized we know that the variety will be generically irreducible.

In [23]:
N_3=matrix(QQ,[1 1 1 0 0 0 0 0 0; 0 0 0 1 1 1 0 0 0;0 0 0 0 0 0 1 1 1])

B_3=matrix(ZZ,[1 0 0 1 0 0 0 0 0; 0 1 0 0 1 0 0 0 0; 0 1 0 0 1 0 1 0 0; 0 1 0 0 1 0 0 2 0])


[1   0   0   1   0   0   0   0   0]
[0   1   0   0   1   0   0   0   0]
[0   1   0   0   1   0   1   0   0]
[0   1   0   0   1   0   0   2   0]

In [24]:
main(N_3,B_3)

"Supportive evidence that the conjecture is true: all square subsystems have generic root count 1 and the variety is generically irreducible"

# Experiment for systems with no square subsystems

In [25]:
function generation(r1::Int64,r2::Int64,r3::Int64,MaxExp::Int64,range::Int64)
    r=r1+r2+r3
    s=2
    n=3
    b_1=rand(0:MaxExp,r)
    b_2=rand(0:MaxExp,r)
    b_3=rand(0:MaxExp,r)
    N=zeros(Int,2,r+2)
    B=zeros(Int,3,r+2)
    N[1,1]=1
    N[2,2]=1
    N[1,3:r1+2]=ones(Int,r1)
    N[2,2+r1+1:2+r1+r2]=ones(Int,r2)
    N[2,2+r1+r2+1:2+r1+r2+r3]=rand(-range:range,r3)
    N[1,2+r1+r2+1:2+r1+r2+r3]=ones(Int,r3)
    B[1,3:r+2]=b_1
    B[2,3:r+2]=b_2
    B[3,3:r+2]=b_3
    return [matrix(QQ,N),matrix(ZZ,B)]
end 



generation (generic function with 1 method)

In [26]:
function no_square_subsystems_check(N,B) 
        if (!(rank(B)==3))
                return false
       else
               f=system_OS_ones(N,B)
               for i in 1:2
                       exponent_vectors=collect(exponents(f[i]))
                        if (rank(hcat(exponent_vectors...))==1)
                                return false
                        end
               end 

       end 

       return true
end


no_square_subsystems_check (generic function with 1 method)

In [27]:
function test(r1::Int64,r2::Int64,r3::Int64,MaxExp::Int64,range::Int64,m::Int64)
    for i in 1:m
        L=generation(r1,r2,r3,MaxExp,range)
          if (compute_rank(L[1],L[2])==size(L[1])[1])
            if no_square_subsystems_check(L[1],L[2])
                if (irred_check_numerical(L[1],L[2])==false)
                    return [L[1],L[2]]
                end
            end
        end
    end 
return "No candidate for counterexample found"
end 

test (generic function with 1 method)

In [29]:
L=test(2,2,2,2,10,20)

2-element Vector{MatElem}:
 [1 0 1 1 0 0 1 1; 0 1 0 0 1 1 -6 -9]
 [0 0 2 1 1 0 2 1; 0 0 1 0 1 1 1 2; 0 0 2 1 1 1 2 0]

In [30]:
irred_check(L[1],L[2])

true

In [31]:
irred_check_numerical(L[1],L[2])

true